# Bibliotecas

In [ ]:
import os
import json

from sklearn.cluster import KMeans
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate

from dotenv import load_dotenv

In [ ]:
# Carrega as variáveis de ambiente
load_dotenv()

# Verifica se a OPENAI_API_KEY está configurada
assert "OPENAI_API_KEY" in os.environ, "Defina a variável de ambiente OPENAI_API_KEY!"

# Guarda a API key (opcional, já está no env)
openai_api_key = os.getenv("OPENAI_API_KEY")

# Problema

Temos diversos documentos falando sobre diversos tópicos. O objetivo é agrupar esses documentos por temáticas.

Aqui vamos usar LangChain (framework comum) com modelos da OpenAI para fazer o embedding do banco vetorial, e também para criar um label para os grupos.

# Dados Base

In [ ]:
docs = [
    "Python vs JavaScript: An in-depth comparison of two programming languages.",
    "10 tips for Python coding beginners.",
    "How to bake the perfect sourdough bread at home.",
    "Mastering JavaScript closures and callbacks.",
    "Sourdough starter maintenance and bread recipes.",
]

# Solução
A ideia vai ser:
1. Transformar os documentos num banco vetorial (embeddings)
2. Fazer uma clusterização simples nos vetores (k-means)
3. Nomear os clusters com uma LLM

## Carregando modelos
Vamos usar:
- `gpt-4o-mini (OpenAI)` para rodar via API o labeling.
- `text-embedding-3-small (OpenAI)` para fazer o embedding.

In [ ]:
# Inicializando o LLM para rotular clusters
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

In [ ]:
# Inicializando embeddings da OpenAI
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

## Construindo o Banco Vetorial

In [ ]:
doc_vectors = embedding_model.embed_documents(docs)
for vector in doc_vectors:
    print(vector)  # São um vetor para cada documento

## Clusterizando com K-Means
Vamos clusterizar os vetores em 2 grupos.

In [ ]:
# Rodando o k-means
num_clusters = 2  # Assumindo que já sabemos que há 2 grupos
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(doc_vectors)

In [ ]:
# Montando o resultado
clusters = {i: [] for i in range(num_clusters)}
for doc, lab in zip(docs, labels):
    clusters[lab].append(doc)

print("Clusters brutos:")
print(json.dumps(clusters, indent=4))  # Só pra ficar mais legível

## Rotulando os Clusters


In [ ]:
prompt = PromptTemplate.from_template(
    "Here are some documents about the same topic:\n\n"
    "{documents}\n\n"
    "Provide a short label with one or two words, "
    "describing their common theme, all lower case and with only letters and space:"
)

In [ ]:
# Setando a corrente
chain = prompt | llm

In [ ]:
# Fazendo o labeling de cada cluster e substituindo o nome
labeled_clusters = {}
for cid, docs_in_cluster in clusters.items():
    # join cluster docs into bullet list
    docs_text = "\n".join(f"- {d}" for d in docs_in_cluster)
    label = chain.invoke(input={"documents": docs_text}).content
    labeled_clusters[label] = clusters[cid]
    print(f"Cluster {cid} label: {label}")

In [ ]:
print("Clusters rotulados:")
print(json.dumps(labeled_clusters, indent=4))